In [ ]:
import scanpy as sc
import pandas as pd
import anndata as ad
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
from datetime import datetime, date, time
from scipy.spatial.distance import pdist, squareform, cdist
import scipy.integrate as integrate
from scipy.io import savemat
import os
import h5py
import re
import cProfile
import pstats
import time
from scipy.spatial.distance import pdist, squareform
from joblib import Parallel, delayed
import plotly.express as px

In [ ]:
def h_samp(rm, D, N):
   dirs = np.random.normal(size=(N,D))
   dirs = dirs/np.sqrt(np.sum(np.square(dirs), axis=1)).reshape(-1,1)
   r_p = np.asarray([sample_r(rm, D) for i in np.arange(N)])
   return np.tanh(r_p/2.0).reshape(-1,1)*dirs

# sample radial coordinate of point uniformly out to max rad R
def sample_r(R, D):
   rmin = 0.0; rmax = R; rc = 0.5*R
   V_tot = vol(R, D); uc = vol(rc, D)/V_tot
   u = np.random.uniform()
   while(np.abs(u-uc)>1e-4):
       if uc < u:
           rmin = rc
           rc = 0.5*(rc + rmax)
           uc = vol(rc, D)/V_tot
       elif u < uc:
           rmax = rc
           rc = 0.5*(rc + rmin)
           uc = vol(rc, D)/V_tot
   return rc

def poincare_distance(p1, p2):
   """
   Return Hyperbolic distance between poincare coordinates of 2 points
   """
   inv = 2.0*(p1-p2).dot(p1-p2)/((1-p1.dot(p1))*(1-p2.dot(p2)))
   return np.arccosh(1.0 + inv)

def vol(R, D):
   c = 2*np.power(np.pi, 0.5*D)
   itg = integrate.quad(lambda x: np.power(np.sinh(x), D-1), 0.0, R)[0]
   return c*itg

# Get distances based off Poincare coordinates
def get_poincare_dmat(pts):
   """
   Return NxN distance matrix from a collection of points in their poincare coordinates
   """
   N = pts.shape[0]
   dm = np.zeros(shape=(N,N))

   for i in np.arange(N):
      for j in np.arange(i+1, N):
         dm[i][j] = poincare_distance(pts[i], pts[j])
         dm[j][i] = dm[i][j]

   return dm



# Creating sample hyperbolic point samples
def h_samp(rm, D, N):
   dirs = np.random.normal(size=(N,D))
   dirs = dirs/np.sqrt(np.sum(np.square(dirs), axis=1)).reshape(-1,1)
   r_p = np.asarray([sample_r(rm, D) for i in np.arange(N)])
   return np.tanh(r_p/2.0).reshape(-1,1)*dirs



def get_dmat(X, geometry):
    if geometry == 'hyperbolic':
        """
        Return NxN distance matrix from a collection of points in their poincare coordinates
        """
        N = X.shape[0]
        dm = np.zeros(shape=(N,N))
    
        for i in np.arange(N):
           for j in np.arange(i+1, N):
              dm[i][j] = poincare_distance(X[i], X[j])
              dm[j][i] = dm[i][j]
    
        return dm
    if geometry == 'euclidean':
        return squareform(pdist(X, geometry))



def batch_pdist(X, batch_size):
    n_samples = X.shape[0]
    dm = np.zeros((n_samples, n_samples), dtype=np.float64)
    
    for start in range(0, n_samples, batch_size):
        end = min(start + batch_size, n_samples)
        print(f"Processing batch: {start} to {end}")
        
        # Compute pairwise distances within the batch
        batch_dm = pdist(X[start:end], 'euclidean')
        batch_dm_square = squareform(batch_dm)
        dm[start:end, start:end] = batch_dm_square
        
        # Compute distances between this and all other batches
        for other_start in range(0, n_samples, batch_size):
            print("Other start",other_start)
            other_end = min(other_start + batch_size, n_samples)
            if start != other_start:  # Avoid computing the same batch twice
                inter_batch_dm = cdist(X[start:end], X[other_start:other_end], 'euclidean')
                dm[start:end, other_start:other_end] = inter_batch_dm
                dm[other_start:other_end, start:end] = inter_batch_dm.T

    return dm

def create_random_matrix(rows, cols, low=0, high=100):
    return np.random.randint(low, high, size=(rows, cols))

In [ ]:
'''
Final code takes in anndata checks if it has information in obsp and if it doesn't creates a 
distance matrix based off main data and places it there to later be used in hyperbolic embedding
'''
file_path = '/Users/anthony/Documents/vscode/Hyperbolic-MDS/AnnData/fMDSdata.h5ad'
obs_metadata_path = "/Users/anthony/Documents/vscode/Hyperbolic-MDS/Saved_data/E-MTAB-62.sdrf.txt"
parts_dir = '/Users/anthony/Documents/vscode/Hyperbolic-MDS/AnnData/chunks/'
part_files = sorted([f for f in os.listdir(parts_dir) if f.startswith('fakeDataMatrix_part')],
                    key=lambda x: int(re.findall(r'\d+', x)[0]))


adata = ad.read_h5ad(file_path)


if len(adata.obsp) == 0:
    distance_matrix = batch_pdist(adata.X, batch_size=500)
    print("shape of distance matrix based off intensity data",distance_matrix.shape)
    print(distance_matrix.dtype)
    adata.obsp['distance_matrix'] = distance_matrix
    print("Size", adata.obsp['distance_matrix'].shape)
    print("Size",adata.obsp.shape)
    print(adata.obsp.head(10))
else:
    print("Already has pairwise observation data")